### Python 代码运行并测试运行时间

In [21]:
import numpy as np
from time import time

rng = np.random.default_rng()


def measure_runtime(func, *args, **kwargs):
    start_time = time()
    result = func(*args, **kwargs)
    runtime = time() - start_time
    return result, runtime


def RandomData(n, p):
    mean_vec = np.zeros(p)
    cov_mat = 0.5 ** np.abs(np.arange(1, p + 1).reshape(-1, 1) - np.arange(1, p + 1))
    X = rng.multivariate_normal(mean_vec, cov_mat, n, method="cholesky")
    return X


def MatrixMultiplication(p):
    a = rng.normal(size=(p, p))
    b = a.T @ a
    return b


def DistributedRandom(n, p, m):
    mean_vec = np.zeros(p)
    cov_mat = 0.5 ** np.abs(np.arange(1, p + 1).reshape(-1, 1) - np.arange(1, p + 1))
    x = [0] * m
    for i in range(m):
        x[i] = rng.multivariate_normal(mean_vec, cov_mat, n, method="cholesky")
    return x


# Example usage:
Data_result, Time_data = measure_runtime(RandomData, n=200, p=2000)
print(f"随机数生成结果: {Data_result.shape}")
print(f"随机数生成时间: {Time_data} seconds")

Matrix_result, Time_matrix = measure_runtime(MatrixMultiplication, p=2000)
print(f"矩阵乘法结果: {Matrix_result.shape}")
print(f"矩阵乘法时间: {Time_matrix} seconds")


Distributed_result, Distributed_time = measure_runtime(DistributedRandom, n=6000, p=600, m=60)
print(f"分布式随机数生成时间: {Distributed_time} seconds")


随机数生成结果: (200, 2000)
随机数生成时间: 0.08186912536621094 seconds
矩阵乘法结果: (2000, 2000)
矩阵乘法时间: 0.07513713836669922 seconds
分布式随机数生成时间: 3.210928201675415 seconds
